In the article by Hu & Bentley 2000, the authors defined $j$ as the number of virions infecting a cell. The infection probability was described by a Poisson distribution:

$P(t, j) = \frac{exp(-dynMOI) \times dynMOI^j}{j!}$

To simplify calculation, the authors proposed setting a range of $j$ by covering 99% of possible infection events. However, the authors didn't provide the method of defining the range. Here I'm going to implement the range calculation. Use the `scipy.stats.poisson` for cumulative distribution function (CDF) and probability mass function (pmf) of Poission distribution. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import poisson

In [ ]:
dynMOI_range = (0.04*0.1, 500)
dynMOI_range

In [ ]:
def j_range_poisson(mu, pct=0.99):
    """
    Find the shortest closed intervel [jmin, jmax] that covers 
    >= pct events in a poisson distribution defined by mu.
    """
    # set range, calculate cdf and pmf
    span = round(mu+1)*2 + 1
    assert poisson.cdf(span-1, mu) > pct
    
    # 2 pointers, beginning with round(mu)
    pmf = poisson.pmf(range(0, span), mu)
    jmin = jmax = round(mu)
    cur_pct = sum(pmf[jmin:(jmax+1)])
    while (
        cur_pct <= pct
        and jmin >= 0
        and jmax <= span-1
    ):
        if pmf[jmin-1] >= pmf[jmax+1]:
            cur_pct += pmf[jmin-1]
            jmin -= 1
        else:
            cur_pct += pmf[jmax+1]
            jmax += 1

    if jmin == -1:    # reaches left most
        jmin = 0
        while (cur_pct <= pct) and (jmax < span-1):
            jmax += 1
            cur_pct += pmf[jmax]
    if jmax == span:    # reaches right most
        jmax = span-1
        while (cur_pct <= pct) and (jmin > 0):
            jmin -= 1
            cur_pct += pmf[jmin]
    return jmin, jmax

Compare with the empirical linear range used in `baculo_vlp` module.

In [ ]:
def jmin_jmax(alpha: float, V1: float, N: float):
    """Upper and lower bound of infecting virus number j.
    Args:
        alpha:
        V1:
        N:
    """
    # extreme cases: no virus or no cells
    if V1 == 0 or int(N) == 0:
        return 0, 0
    
    dynMOI = alpha * V1 / N
    # jmin
    if dynMOI < 20:
        jmin = 0
    elif dynMOI >= 20 and dynMOI < 480:
        jmin = round(dynMOI - 20)
    else:
        jmin = 460
    
    # jmax
    if dynMOI < 2.5:
        jmax = 6
    elif dynMOI >= 2.5 and dynMOI < 5:
        jmax = 12
    elif dynMOI >= 5 and dynMOI < 480:
        jmax = round(dynMOI + 20)
    else:
        jmax = 500

    return jmin, jmax

In [ ]:
from functools import partial

j_range_emp = partial(jmin_jmax, alpha=1, N=1)

Since an insect cell has a maximum of 11,000 virus receptors, and the $alpha$ is 0.04 or 0.08, it is reasonable to assume the dynMOI bigger than ~500 doesn't make a difference.

In [ ]:
from itertools import chain, product

In [ ]:
dynMOIs = list(chain(
    [0.004, 0.1, 0.5, 1], 
    range(2, 11, 2), 
    range(20, 100, 10), 
    range(100, 600, 100)
))
df = pd.DataFrame(
    index=dynMOIs, 
    columns=[
        "_".join(tup) for tup in product(
            ("stat", "emp"), ("jmin", "jmax")
        )
    ])

In [ ]:
for dynmoi in dynMOIs:
    (
        df.loc[dynmoi, "stat_jmin"], 
        df.loc[dynmoi, "stat_jmax"]
    ) = j_range_poisson(dynmoi)
    (
        df.loc[dynmoi, "emp_jmin"], 
        df.loc[dynmoi, "emp_jmax"]
    ) = j_range_emp(V1=dynmoi)

In [ ]:
plt.plot(df["stat_jmin"].reset_index(drop=True), color="blue", linestyle="-")
plt.plot(df["stat_jmax"].reset_index(drop=True), color="blue", linestyle="-")
plt.plot(df["emp_jmin"].reset_index(drop=True), color="grey", linestyle="--")
plt.plot(df["emp_jmax"].reset_index(drop=True), color="grey", linestyle="--")
plt.xticks(range(0, df.shape[0]), df.index, rotation=45)
plt.legend(df.columns)
plt.xlabel("dynMOI")
plt.ylabel("j")

For dynMOI at 4.0~60 range, the empirical range of j is wider than statistical. However, for dynMOI > 100, the statistical range is wider, which means our empirical j range is not sufficient.

In [ ]:
def j_range_newemp(dynmoi):
    if dynmoi < 1:
        return 0, 4
    elif dynmoi < 10:
        return 0, round(dynmoi+1)*2
    elif dynmoi < 60:
        return max(0, round(dynmoi - 20)), round(dynmoi + 20)
    elif dynmoi < 500:
        rad = 0.1*dynmoi + 16
        return round(dynmoi - rad), round(dynmoi + rad)
    else:
        return 440, 560

In [ ]:
df[["newemp_jmin", "newemp_jmax"]] = [
    j_range_newemp(dynmoi) for dynmoi in dynMOIs
]

In [ ]:
plt.plot(df["stat_jmin"].reset_index(drop=True), color="blue", linestyle="-")
plt.plot(df["stat_jmax"].reset_index(drop=True), color="blue", linestyle="-")
plt.plot(df["newemp_jmin"].reset_index(drop=True), color="grey", linestyle="--")
plt.plot(df["newemp_jmax"].reset_index(drop=True), color="grey", linestyle="--")
plt.xticks(range(0, df.shape[0]), df.index, rotation=45)
plt.legend(["stat_jmin", "stat_jmax", "newemp_jmin", "newemp_jmax"])
plt.xlabel("dynMOI")
plt.ylabel("j")

This one is more similar to statistical deduction. When using the new j range, much of the authors' conclusions are not changed but the calculation became a little bit slower (since the range is wider than before). 

Note:  
The **upper limit of j** does play a very important role in making the simulation speed reasonable. 